In [66]:
def guruswami_sudan(u, C, m, l, r):
    """
    Decode the noisy codeword u with code C using the Guruswami Sudan method.
    This method implements a list decoding algorithm to retrieve a list of size at most l.

    Input :
        - u : the noisy codeword to decode
        - C : a Reed Solomon code
        - m : a decoding parameter
        - r : the max number of errors in the encoded word (to compare to floor((n-k)/2))
                asymptoticaly n - sqrt(2k*n)
        - l : the maximum size of the list of possible codewords (l_max = ceil(m(n-r)/(k-1) - 1))
    Output :
        - x : a list of possible messages
    """

    # Retrieve the code parameters
    n = C.length()
    k = C.dimension()
    F = C.base_field()
    x = C.evaluation_points()

    # Create the variables and polynomial rings
    # The number of variables is sum[j=1..l](m*(n-r) - j(k-1))
    nb_unknowns = (l + 1) * m * (n - r) - (k - 1) * ((l * (l + 1))/2)
    # To define many variables in F we need a sagemath trick
    unknowns = PolynomialRing(F, nb_unknowns, "a")
    # The polynomials are bivariate over F
    R.<X,Y> = unknowns[]

    # Compute Q as the sum of Q_j polynomials
    Q = 0 * X * Y
    offset = 0
    for j in range(l+1):
        deg = m*(n-r) - j*(k-1) - 1
        Q += sum([
                unknowns.gen(i + offset) * X^i for i in range(deg + 1)
            ]) * Y^j
        offset += deg + 1

    # Collect all the equations corresponding to the multiplicity of (x_i, f(x_i))
    # meaning that there are no monomials of degree < m
    equations = []
    for i in range(n):
        Q_i = Q(X + x[i], Y + u[i])
        # get all the monomials of deg < m
        for d_x in range(m):
            for d_y in range(m-d_x):
                coef = Q_i[d_x, d_y]
                equations.append([coef.coefficient({unknowns.gen(i): 1}) for i in range(nb_unknowns)])
    # Solve the system of equation using matrix reduction
    equations = matrix(F, equations)
    S = equations.right_kernel().basis()[0]

    # S holds the coefficients of Q
    # Rebuild the polynomial as a polynomial with coefficients in F[X] and solve for Y
    R = PolynomialRing(F, "X")
    RR.<Y> = R[]
    # Rebuild Q
    offset = 0
    X_coefs = []
    for j in range(l+1):
        deg = m*(n-r) - j*(k-1) - 1
        X_coefs.append(sum([
                S[i + offset] * X^i for i in range(deg + 1)
            ]))
        offset += deg + 1
    P = RR(X_coefs)
    # Compute and return the roots, containing the encoded message
    f = [p[0].coefficients(sparse=false) for p in P.roots()]
    return(f)

In [67]:
q = 2^8
n = 32
k = floor(1/3 * n)
y = [1] * n # Normal Reed Solomon code
F.<g> = GF(q)
C = codes.GeneralizedReedSolomonCode(F.list()[:n], k, y)

# For Guruswami Sudan decoding two parameters are needed :
# - m : the multiplicity parameter. Because the complexity is in m^6, we can only take small values of m.
#       For testing, we take m = 3
# - l : the maximum length of the decoding list.
# Asymptotically, the number of errors to correct can be up to n - sqrt(2k*n)
m = 3
# The radius and the best l parameter can be computed with sage
GSD = codes.decoders.GRSGuruswamiSudanDecoder
r, (m, l) = GSD.guruswami_sudan_decoding_radius(C, s=m)

msg = random_vector(C.base_field(), C.dimension())

print(f"Original msg    = {msg}")
u = C.encode(msg)
print(f"Encoded msg u   = {u}")
chan = channels.StaticErrorRateChannel(C.ambient_space(), r)
v = chan(u)
print(f"With {(u - v).hamming_weight()} errors u = {v}")

decoded = guruswami_sudan(v, C, m=m, l=l, r=r)
print(f"Decoded list [{len(decoded)}] = {decoded}")

Original msg    = (g^7 + g^5 + g^4 + g^3 + g + 1, g^5 + g^4 + g^3 + g^2 + g, g^7 + g^6 + g^4 + g^3 + g + 1, g^7 + g^4 + g^3 + g^2, g^6 + g^3 + g^2 + 1, g^4 + g^3 + g + 1, g^6 + g^5 + g^3 + g^2 + 1, g^7 + g^5 + g^4 + g^3 + g^2 + 1, g^5 + g^3 + g^2 + g, g^5 + g^3 + g^2)
Encoded msg u   = (g^7 + g^5 + g^4 + g^3 + g + 1, g^6 + g^4 + g^2 + g, g^5 + g^2 + g, g^7 + g^6 + g^2, g^7 + g^5 + g^4 + g^3 + g^2 + g + 1, g^5 + g^4 + 1, g, g^6 + g^4 + g^3 + g^2 + g + 1, g^6 + g^3, g^7 + g^6 + g^4 + g^2 + g + 1, g^6 + g^4 + g^3 + g + 1, g^5 + g^3 + g^2 + g, g^5 + g^3, g^7 + g^6 + g^5 + g^3 + g + 1, g^6 + g^5 + g^4 + g^3 + g^2 + g + 1, g^6 + g^5 + g^3 + g^2 + g, g^7 + g^6 + g^2, g^6 + g^5 + g^4 + 1, g^7 + g^6 + g^5 + g^4, g, g^7 + g^6 + g^5 + g^4 + g^2 + g, g^7 + g^5 + g^3 + g, g^6 + g^3 + g^2 + g + 1, g^6, g^6 + g^5 + g^3 + g + 1, g^6 + g^4 + g^3 + g^2 + g, g^5 + g^3 + g^2 + g + 1, g^7 + g^6 + g^5 + g, g^3 + g^2 + g + 1, g^7, g^7 + g^6, g^5 + g^4 + g^3)
With 13 errors u = (g^7 + g^5 + g^4 + g^3 + g + 1,